In [1]:
import numpy as np
import tensorflow as tf
import tflearn
import random
import nltk
# from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
#Sentence Tokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
# Open intent JSON file
with open('intents.json') as json_intent:
    intents = json.load(json_intent)

In [4]:
# Create three lists to strore words,classes and documents
words = []
classes = []
documents = []
ignore_words='''!()-[]{};:'"\,<>./?@#$%^&*_~'''

In [5]:
# Add the tokenized words from each intent into words list
# Create document by adding adding each word and its tag 
# Create list of all different tag (classes) 
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
# Modify the words list by stroring the stem words
# Also removing ignore words
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [7]:
# Creating Training data and output data
training = []
output = []
output_empty = [0] * len(classes)

# Create doc (bag) for each sentence. If the same value of doc is also in words list add '1' else add '0'.
#Append bag and output row in the traing list
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [8]:
import tensorflow as tf
random.shuffle(training)
training = np.array(training)

In [9]:
# Divide input and output data
train_x = list(training[:,0])
train_y = list(training[:,1])

In [10]:
from tensorflow.python.framework import ops
ops.reset_default_graph() 

In [11]:
# Created TF fully connected feed forward neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
# Train the model and saved it.
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.37939
| Adam | epoch: 1000 | loss: 0.37939 - acc: 0.9601 -- iter: 24/27
Training Step: 4000  | total loss: 0.34659 | time: 0.008s
| Adam | epoch: 1000 | loss: 0.34659 - acc: 0.9641 -- iter: 27/27
--
INFO:tensorflow:C:\Users\harsh\OneDrive\Desktop\Desktop\JOB\Projects\Chatbot\AI_ChatBot_Python-master\AI_ChatBot_Python-master\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [13]:
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [14]:
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

with open('intents.json') as json_data:
    intents = json.load(json_data)

In [15]:
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\harsh\OneDrive\Desktop\Desktop\JOB\Projects\Chatbot\AI_ChatBot_Python-master\AI_ChatBot_Python-master\model.tflearn


In [16]:
# Defined Functions of each process
# Takenization and Stemming
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Bag of Words
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

# Defined Confidence Threshold
ERROR_THRESHOLD  = 0.25

# Predict the sentence class
def classify(sentence):
    results = model.predict([bow(sentence, words)])[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD ]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

In [17]:
# Generate response
def response(sentence,userID='123', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    return print(random.choice(i['responses']))
            results.pop(0)

In [ ]:
# Lets Chat
while True:
    input_data = input("You- ")
    answer = response(input_data)
    answer

You- hi
Hi there, how can I help?
You- open 
We're open every day from 9am-9pm
